In [1]:
import tensorflow as tf 

tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [20]:
import tensorflow as tf
import cv2
import numpy as np
import numpy  as np
from PIL import Image
import os
import time

from configuration import Config
from core.efficientdet import EfficientDet, PostProcessing
from data.dataloader import DataLoader

In [31]:
# 원래 모델

def idx2class():
    return dict((v, k) for k, v in Config.pascal_voc_classes.items())


def draw_boxes_on_image(image, boxes, scores, classes):
    num_boxes = boxes.shape[0]
    for i in range(num_boxes):
        class_and_score = str(idx2class()[classes[i]]) + ": " + str(scores[i])
        cv2.rectangle(img=image, pt1=(boxes[i, 0], boxes[i, 1]), pt2=(boxes[i, 2], boxes[i, 3]), color=(255, 0, 0), thickness=2)
        cv2.putText(img=image, text=class_and_score, org=(boxes[i, 0], boxes[i, 1] - 10), fontFace=cv2.FONT_HERSHEY_COMPLEX, fontScale=1.5, color=(0, 255, 255), thickness=2)
    return image

def test_single_picture(picture_dir, model):
    image_array = cv2.imread(picture_dir)
    image = DataLoader.image_preprocess(is_training=False, image_dir=picture_dir)
    image = tf.expand_dims(input=image, axis=0)

    outputs = model(image, training=True)
    post_process = PostProcessing()
    boxes, scores, classes = post_process.testing_procedure(outputs, [image_array.shape[0], image_array.shape[1]])
#     print("-"*50)
#     print("boxes")
#     print(boxes)
#     print("-" * 50)
#     print("scores")
#     print(scores)
#     print("-" * 50)
#     print("classes")
#     print(classes)
#     print("-" * 50)
    image_with_boxes = draw_boxes_on_image(image_array, boxes.astype(np.int), scores, classes)
    return image_with_boxes

def test_batch_picture(image_list, model):
    outputs = model(image_list_batch, training=True)
    post_process = PostProcessing()
    boxes = []
    scores = []
    classes = []
    for i in range(32):
        print(outputs[i].shape)
        print(image_list[i].shape)
        image  = image_list[i]
        image  = tf.expand_dims(input=image, axis=0)     
        boxes[i], scores[i], classes[i] = post_process.testing_procedure(outputs[i], [ image.shape[0], image.shape[1]])
    
    print("boxes : ", boxes.shape)
    print("scores : ", scores.shape)
    print("classes : ", classes.shape)
    image_with_boxes = draw_boxes_on_image(image_array, boxes.astype(np.int), scores, classes)
    return image_with_boxes

def data_load(image_path):
    path = image_path
    count = 0
    image_list = []
    for item in os.listdir(path)[:32]:
        imgpath = path +'/' + item
        img = Image.open(imgpath)
        arr = np.array(img)
        image_list.append(arr)
        #img.save(str(count) +  "output.png")
        count = count +1
    image_batch =  tf.image.resize(image_list, (512,512))
#     image_temp = tf.data.Dataset.from_tensor_slices(image_list).batch(32).take(1)
#     image_batch =  tf.image.resize(image_temp, (512,512))
    return image_batch



if __name__ == '__main__':
    # GPU settings
    gpus = tf.config.list_physical_devices("GPU")
    if gpus:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
            
            
    load_weights_from_epoch = Config.load_weights_from_epoch_quan
    efficientdet = EfficientDet()
    #efficientdet.load_weights(filepath=Config.save_model_dir + "saved_model")
 
    efficientdet.load_weights(filepath=Config.save_model_dir+"epoch-{}".format(load_weights_from_epoch))
    
    
    image_path = './data/fire_smoke/train/JPEGImages'
    image_list_batch =  data_load(image_path)
    
    t1 = time.time()
    outputs = efficientdet(image_list_batch, training=True)
    t2 = time.time()
    print("------------------------------------")
    print(t2-t1)
    print(outputs.shape)
    
    
    image_output_list = test_batch_picture(image_list_batch, efficientdet)
    print(image_output_list.shape)
    
    
    test_image_dir = "./data/fire_smoke/test/JPEGImages/ck0kdhymna0b10721v4wntit8_jpeg.rf.a08e34d04fb672ce6cf8e94e810ec81d.jpg"
    image = test_single_picture(picture_dir=test_image_dir, model=efficientdet)

#     cv2.namedWindow("detect result", flags=cv2.WINDOW_NORMAL)
#     cv2.imshow("detect result", image)
#     cv2.waitKey(0)

(32, 512, 512, 3)
------------------------------------
2.3264358043670654
(32, 49104, 5)
(49104, 5)
(512, 512, 3)


IndexError: too many indices for array: array is 2-dimensional, but 3 were indexed

In [ ]:
cv2.imshow("detect result", image)
cv2.waitKey(0)

In [6]:
# tflite 평가 하기

def idx2class():
    return dict((v, k) for k, v in Config.pascal_voc_classes.items())


def draw_boxes_on_image(image, boxes, scores, classes):
    num_boxes = boxes.shape[0]
    for i in range(num_boxes):
        class_and_score = str(idx2class()[classes[i]]) + ": " + str(scores[i])
        cv2.rectangle(img=image, pt1=(boxes[i, 0], boxes[i, 1]), pt2=(boxes[i, 2], boxes[i, 3]), color=(255, 0, 0), thickness=2)
        cv2.putText(img=image, text=class_and_score, org=(boxes[i, 0], boxes[i, 1] - 10), fontFace=cv2.FONT_HERSHEY_COMPLEX, fontScale=1.5, color=(0, 255, 255), thickness=2)
    return image

def load_model(path):  
    interpreter = tf.lite.Interpreter(model_path = path)
    interpreter.allocate_tensors()
    return interpreter

def test_single_picture(picture_dir, model):
    image_array = cv2.imread(picture_dir)
    image = DataLoader.image_preprocess(is_training=False, image_dir=picture_dir)
    image = tf.expand_dims(input=image, axis=0)
    
    input_data = np.array(image, dtype = np.uint8)
    input_details = model.get_input_details()[0]
    output_details = model.get_output_details()[0]
    model.set_tensor(input_details['index'], input_data)
    model.invoke()
    outputs= model.get_tensor(output_details['index'])
    
    print(outputs)
    # 원래 outputs =  소수점 형태
    # quantizaation model =  숫자 너무큼.. output_tensor shape 해결하기
    
    
    
    outputs = tf.constant(outputs)
    
    post_process = PostProcessing()
    boxes, scores, classes = post_process.testing_procedure(outputs, [image_array.shape[0], image_array.shape[1]])
    print("-"*50)
    print("boxes")
    print(boxes)
    print("-" * 50)
    print("scores")
    print(scores)
    print("-" * 50)
    print("classes")
    print(classes)
    print("-" * 50)
    image_with_boxes = draw_boxes_on_image(image_array, boxes.astype(np.int), scores, classes)
    return image_with_boxes

test_image_dir = "./data/fire_smoke/test/JPEGImages/ck0kdhymna0b10721v4wntit8_jpeg.rf.a08e34d04fb672ce6cf8e94e810ec81d.jpg"

gpus = tf.config.list_physical_devices("GPU")

if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)

model_path = "./tflite_model_quant.tflite"
efficientdet_lite = load_model(model_path)

image = test_single_picture(picture_dir=test_image_dir, model=efficientdet_lite)

cv2.namedWindow("detect result", flags=cv2.WINDOW_NORMAL)
cv2.imshow("detect result", image)
cv2.waitKey(0)

ValueError: Cannot set tensor: Dimension mismatch. Got 1 but expected 32 for dimension 0 of input 0.

In [1]:
!nvidia-smi

Tue Oct 26 16:12:56 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 470.57.02    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA TITAN RTX    Off  | 00000000:AF:00.0 Off |                  N/A |
| 41%   35C    P8    25W / 280W |  23734MiB / 24220MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------